This is an interactive example that will walk you through the initialization of a RAG and the basic embedding of a few documents.

In [ ]:
from pathlib import Path
import os

from backend.config import RagConfig
from backend.rag_components.rag import RAG

repo_root = Path(os.getcwd())

rag_config = RagConfig.from_yaml(repo_root / "backend" / "config.yaml")
rag = RAG(config=rag_config)

print("LLM:", rag.llm.__class__.__name__)
print("Embedding model:", rag.embeddings.__class__.__name__)
print("Vector store:", rag.vector_store.__class__.__name__)
print("Retriever:", rag.retriever.__class__.__name__)

Here we transform our CSV into standalone embeddable documents that we will be able to feed the vector store.

We generate one document for each line, and each document will contain header:value pairs for all the columns.

This is a very simplistic example, but vector store data models can get more advanced to support more [powerful retreival methods.](https://python.langchain.com/docs/modules/data_connection/retrievers/)

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores.utils import filter_complex_metadata


data_sample_path = repo_root / "examples" / "billionaires.csv"

loader = CSVLoader(
    file_path=str(data_sample_path),
    csv_args={"delimiter": ",", "quotechar": '"', "escapechar": "\\"},
    encoding="utf-8-sig",
)

raw_documents = loader.load()
documents = filter_complex_metadata(raw_documents)
documents[:5]

To load the docs in the vector store, we recommend using the `load_document` as it [indexes previously embedded docs](https://python.langchain.com/docs/modules/data_connection/indexing), making the process idempotent.

In [ ]:
rag.load_documents(documents)